In [57]:
import pandas as pd
import numpy as np
import cPickle 
from nltk.corpus import stopwords
from gensim.models import word2vec
import nltk.data
import re
import logging
from nltk.stem.snowball import *
import itertools

# Python 2.x:
import HTMLParser
html_parser = HTMLParser.HTMLParser()

import multiprocessing
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)   

from nltk.stem import WordNetLemmatizer, PorterStemmer
wordnet_lemmatizer = WordNetLemmatizer()
porter = PorterStemmer()

In [58]:
TEST_CASE = {}

In [59]:
import re
import HTMLParser
html_parser = HTMLParser.HTMLParser()
from nltk.stem import WordNetLemmatizer, PorterStemmer
wordnet_lemmatizer = WordNetLemmatizer()
porter = PorterStemmer()

def skill_transform(skill, remove_stopwords = True):
    skill = unicode(skill)
    skill = html_parser.unescape(skill)
    
    skill = skill.replace("_", " ").split()
    skill = " ".join([sk for sk in skill if sk])
    
    skill = re.sub(r"\(.*\)", "", skill)
    skill = skill.replace("-", "") \
        .replace(".", "") \
        .replace(",", "") \
        .replace("-", "") \
        .replace(":", "") \
        .replace("(", "") \
        .replace(")", "") \
        .replace(u"åá", "") \
        .replace(u"&", "and") \
        .replace(" js", "js") \
        .replace("-js", "js") \
        .replace("_js", "js") \
        .replace("java script", "js") 
    
    skill = skill.lower()
    
    # Special cases replace
    special_case = {}
    special_case["javascript"] = [ "js", "java script", "javascripts", "java scrip" ]
    special_case["wireframe"] = [ "wireframes", "wire frame", "wire frames", "wire-frame", "wirefram", "wire fram", "wireframing" ]
    special_case["OOP"] = [  "object oriented", "object oriented programming", ]
    special_case["OOD"] = [ "object oriented design", ]
    special_case["OLAP"] = [ "online analytical processing",  ]
    special_case["Ecommerce"] = [ "e commerce",  ]
    special_case["consultant"] = [ "consulting",  ]
    special_case["ux"] = [ "user experience", "web user experience design", "user experience design", "ux designer", "user experience/ux" ]
    special_case["html5"] = [ "html 5",  ]
    special_case["j2ee"] = [ "jee",  ]
    special_case["osx"] = [ "mac os x", "os x" ]
    special_case["senior"] = [ "sr" ]
    special_case["qa"] = [ "quality",  ]
    special_case["bigdata"] = [ "big data",  ]
    special_case["webservice"] = [ "webservices", "website", "webapps" ]
    special_case["xml"] = [ "xml file", "xml schemas", "xml/json", "xml web service" ]
    special_case["bigdata"] = [ "big data",  ]
    special_case["nlp"] = [ "natural language process", "natural language", "nltk" ]
    special_case["aws"] = [ "amazon web service", ]
    
    for root_skill in special_case:
        if skill in special_case[root_skill]:
            skill = root_skill
    
    # Special case regex
    special_case_regex = {
        r'^angular.*$': 'angularjs',
        r'^node.*$': 'nodejs',
        r'^(.*)[_\s]js$': '\\1js',
        r'^(.*) js$': '\\1js',
        r'^(.*) (and|or).*$': '\\1',
    }
    for regex_rule in special_case_regex:
        after_skill = re.sub(regex_rule, special_case_regex[regex_rule], skill)
        if after_skill != skill:
            skill = after_skill
            break
    
    # Stem
    if len(skill) > 2:
        skill_after = skill.split(" ")
        skill_after = [wordnet_lemmatizer.lemmatize(sk, pos="v") for sk in skill_after]
        skill_after = " ".join(skill_after)
        skill = skill_after
    
    # skill stopwords 
    if remove_stopwords:
        skill_stopwords = [ "app", "touch", "the", "application", "programming", "program", "design"
                           "developer", "framework", "development", "programmer", "technologies",
                          "advance", "core"]
        skill_after = skill.split(" ")
        skill = " ".join([ sk for sk in skill_after if sk not in skill_stopwords ])
    
    skill = skill.lower().strip().replace(" ", "_")
    skill = re.sub(' +',' ', skill)
    
    # NOTE: replace js tail
    skill = re.sub('js$','', skill)
    
    return skill

skill_transform("js")
skill_transform("angularjs")

u'angular'

In [60]:
skill_transform("html5_and_css3_or_html6")

u'html5_and_css3'

In [61]:
def test_case(model, model_name):
    cases = [ "machine learning", "js", "javascript", "python", "html", "html5", "css", "angular", "nodejs" ]
    TEST_CASE[model_name] = []
    for case in cases:
        try:
            TEST_CASE[model_name].append(model.similar_by_word(skill_transform(case)))
        except:
            pass

In [62]:
data_must_have = pd.read_csv('mustHaveSkills-2.csv', header = 0, encoding='ISO-8859-1')
data_must_have.drop_duplicates(inplace=True, keep='last')
data_must_have = data_must_have[data_must_have["job_title"] != 0]
del data_must_have["job_brief_id"]
# data_must_have['Count'] = data_must_have.groupby('job_title')['keyword_name'].transform(pd.Series.value_counts)
data_must_have.drop_duplicates(inplace=True)

unique_skill_before = len(data_must_have['keyword_name'].unique())
print "Before: %s" % unique_skill_before

data_must_have['skill'] = data_must_have['keyword_name'].apply(skill_transform)

unique_skill_after = len(data_must_have['skill'].unique())
print "After: %s" % unique_skill_after 

print "Resize: %s%%" % (unique_skill_after * 100.0 / unique_skill_before)

# data_must_have['keyword_name'] = data_must_have['keyword_name'].str.replace(' ' ,'_')
# data_must_have

Before: 8589
After: 6198
Resize: 72.1620677611%


In [63]:
# pd.set_option('display.max_rows', 1500)
# data_must_have.drop_duplicates(subset=["keyword_name", "skill"])[["keyword_name", "skill"]].sort_values("skill")[3000:]
musthave_data = data_must_have.groupby('job_title')['skill'].apply(list).tolist()

print ("Training model...")
musthave_model = word2vec.Word2Vec(musthave_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
          size=300, 
          min_count=1, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
          iter=4,
          sg =1
    )

musthave_model.init_sims(replace=True)
test_case(musthave_model, "musthave_model")

2017-05-07 10:32:10,451 : INFO : collecting all words and their counts
2017-05-07 10:32:10,453 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-07 10:32:10,481 : INFO : collected 6198 word types from a corpus of 80810 raw words and 5649 sentences
2017-05-07 10:32:10,482 : INFO : Loading a fresh vocabulary
2017-05-07 10:32:10,501 : INFO : min_count=1 retains 6198 unique words (100% of original 6198, drops 0)
2017-05-07 10:32:10,502 : INFO : min_count=1 leaves 80810 word corpus (100% of original 80810, drops 0)
2017-05-07 10:32:10,517 : INFO : deleting the raw counts dictionary of 6198 items
2017-05-07 10:32:10,518 : INFO : sample=0.001 downsamples 35 most-common words
2017-05-07 10:32:10,518 : INFO : downsampling leaves estimated 77128 word corpus (95.4% of prior 80810)
2017-05-07 10:32:10,519 : INFO : estimated required memory for 6198 words and 300 dimensions: 17974200 bytes
2017-05-07 10:32:10,535 : INFO : resetting layer weights
2017-05-07 10:32:10,

Training model...


2017-05-07 10:32:10,971 : INFO : worker thread finished; awaiting finish of 31 more threads
2017-05-07 10:32:10,976 : INFO : worker thread finished; awaiting finish of 30 more threads
2017-05-07 10:32:11,067 : INFO : worker thread finished; awaiting finish of 29 more threads
2017-05-07 10:32:11,098 : INFO : worker thread finished; awaiting finish of 28 more threads
2017-05-07 10:32:11,220 : INFO : worker thread finished; awaiting finish of 27 more threads
2017-05-07 10:32:11,385 : INFO : worker thread finished; awaiting finish of 26 more threads
2017-05-07 10:32:11,519 : INFO : worker thread finished; awaiting finish of 25 more threads
2017-05-07 10:32:11,574 : INFO : worker thread finished; awaiting finish of 24 more threads
2017-05-07 10:32:11,603 : INFO : worker thread finished; awaiting finish of 23 more threads
2017-05-07 10:32:11,662 : INFO : PROGRESS: at 32.37% examples, 98286 words/s, in_qsize 22, out_qsize 1
2017-05-07 10:32:11,665 : INFO : worker thread finished; awaiting fin

In [64]:
with open("prep_data_tokens_underscore_1", "rb") as g:
    data_dice = cPickle.load(g)
dice_data = [ [ skill_transform(d.replace("_", " ")) for d in dice if d is not None ] for dice in data_dice if dice is not None ]


musthave_dice_data = dice_data + musthave_data
musthave_dice_data = [ list(set([ d for d in data if d is not None and len(d) > 0 ])) for data in musthave_dice_data  ]

print len(musthave_dice_data)
musthave_dice_data

print ("Training model...")
musthave_dice_model = word2vec.Word2Vec(musthave_dice_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
          size=300, 
          min_count=1, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
          iter=4,
          sg =1
    )

musthave_dice_model.init_sims(replace=True)
test_case(musthave_dice_model, "musthave_dice_model")

2017-05-07 10:32:22,460 : INFO : collecting all words and their counts
2017-05-07 10:32:22,461 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-07 10:32:22,540 : INFO : PROGRESS: at sentence #10000, processed 216479 words, keeping 11609 word types
2017-05-07 10:32:22,555 : INFO : collected 12535 word types from a corpus of 254463 raw words and 12795 sentences
2017-05-07 10:32:22,556 : INFO : Loading a fresh vocabulary
2017-05-07 10:32:22,588 : INFO : min_count=1 retains 12535 unique words (100% of original 12535, drops 0)
2017-05-07 10:32:22,588 : INFO : min_count=1 leaves 254463 word corpus (100% of original 254463, drops 0)
2017-05-07 10:32:22,617 : INFO : deleting the raw counts dictionary of 12535 items
2017-05-07 10:32:22,618 : INFO : sample=0.001 downsamples 1 most-common words
2017-05-07 10:32:22,618 : INFO : downsampling leaves estimated 254448 word corpus (100.0% of prior 254463)
2017-05-07 10:32:22,619 : INFO : estimated required memory for 1

12795
Training model...


2017-05-07 10:32:22,828 : INFO : training model with 32 workers on 12535 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=10
2017-05-07 10:32:23,955 : INFO : PROGRESS: at 13.80% examples, 133942 words/s, in_qsize 64, out_qsize 0
2017-05-07 10:32:25,019 : INFO : PROGRESS: at 28.65% examples, 131158 words/s, in_qsize 63, out_qsize 1
2017-05-07 10:32:25,995 : INFO : PROGRESS: at 54.61% examples, 180172 words/s, in_qsize 43, out_qsize 2
2017-05-07 10:32:26,976 : INFO : worker thread finished; awaiting finish of 31 more threads
2017-05-07 10:32:26,989 : INFO : worker thread finished; awaiting finish of 30 more threads
2017-05-07 10:32:27,069 : INFO : PROGRESS: at 69.94% examples, 172171 words/s, in_qsize 29, out_qsize 1
2017-05-07 10:32:27,074 : INFO : worker thread finished; awaiting finish of 29 more threads
2017-05-07 10:32:27,118 : INFO : worker thread finished; awaiting finish of 28 more threads
2017-05-07 10:32:27,176 : INFO : worker thread finished; awaitin

In [65]:
# model.similar_by_word(skill_transform('machine learning'))

In [66]:
# model.similar_by_word('python')

In [67]:
musthave_dice_naruki_data = musthave_dice_data
data_naruki = pd.read_csv('naukri_skill_full', header = 0, encoding='ISO-8859-1')
data_naruki.drop_duplicates(subset=['id', 'skill'], keep='last')

data_naruki['skill'] = data_naruki['skill'].apply(skill_transform)
data_naruki_final = data_naruki.groupby('id')['skill'].apply(list)
print len(data_naruki_final)

for skills in data_naruki_final:
    if len(skills) > 2 and skills not in musthave_dice_naruki_data:
        musthave_dice_naruki_data.append(skills)
        
len(musthave_dice_naruki_data)

120107


110660

In [92]:
import cPickle as pickle
pickle.dump( musthave_dice_naruki_data, open( "musthave_dice_naruki_data.p", "wb" ) )

In [68]:
print ("Training model...")
musthave_dice_naruki_model = word2vec.Word2Vec(musthave_dice_naruki_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
          size=300, 
          min_count=3, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
          iter=4,
          sg =1
    )

musthave_dice_naruki_model.init_sims(replace=True)
test_case(musthave_dice_naruki_model, "musthave_dice_naruki_model")

2017-05-07 10:36:14,874 : INFO : collecting all words and their counts
2017-05-07 10:36:14,875 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-07 10:36:14,953 : INFO : PROGRESS: at sentence #10000, processed 216479 words, keeping 11609 word types
2017-05-07 10:36:14,998 : INFO : PROGRESS: at sentence #20000, processed 316511 words, keeping 21557 word types
2017-05-07 10:36:15,038 : INFO : PROGRESS: at sentence #30000, processed 403006 words, keeping 30521 word types


Training model...


2017-05-07 10:36:15,078 : INFO : PROGRESS: at sentence #40000, processed 489603 words, keeping 38336 word types
2017-05-07 10:36:15,119 : INFO : PROGRESS: at sentence #50000, processed 579752 words, keeping 45956 word types
2017-05-07 10:36:15,157 : INFO : PROGRESS: at sentence #60000, processed 666564 words, keeping 52001 word types
2017-05-07 10:36:15,197 : INFO : PROGRESS: at sentence #70000, processed 751300 words, keeping 58857 word types
2017-05-07 10:36:15,236 : INFO : PROGRESS: at sentence #80000, processed 836205 words, keeping 65151 word types
2017-05-07 10:36:15,273 : INFO : PROGRESS: at sentence #90000, processed 923058 words, keeping 70256 word types
2017-05-07 10:36:15,312 : INFO : PROGRESS: at sentence #100000, processed 1013120 words, keeping 74899 word types
2017-05-07 10:36:15,349 : INFO : PROGRESS: at sentence #110000, processed 1099320 words, keeping 80588 word types
2017-05-07 10:36:15,354 : INFO : collected 80695 word types from a corpus of 1105249 raw words and 1

In [69]:
TEST_CASE

{'musthave_dice_model': [[(u'doctoral', 0.9242714047431946),
   (u'postdoc', 0.9213629961013794),
   ('bigdata', 0.9166296124458313),
   (u'doctor', 0.9145913124084473),
   (u'hadoop', 0.9049304127693176),
   (u'dsp', 0.9031425714492798),
   (u'weka', 0.8997036218643188),
   (u'data_science', 0.8971030116081238),
   (u'hdfs', 0.8913494348526001),
   (u'recommendation', 0.89049232006073)],
  [(u'css', 0.9650534987449646),
   (u'css3', 0.9572659134864807),
   (u'html5', 0.949856162071228),
   (u'prototype', 0.9491482377052307),
   (u'react', 0.9414901733398438),
   (u'html5/css3/javascript', 0.9389783143997192),
   (u'html', 0.9377188682556152),
   (u'user_journey', 0.9332085251808167),
   (u'cascade_style_sheet', 0.9284156560897827),
   (u'adobe_aem', 0.9280628561973572)],
  [(u'css', 0.9650534987449646),
   (u'css3', 0.9572659134864807),
   (u'html5', 0.949856162071228),
   (u'prototype', 0.9491482377052307),
   (u'react', 0.9414901733398438),
   (u'html5/css3/javascript', 0.9389783143

In [70]:
TEST_CASE_TABLE = [ pd.DataFrame(TEST_CASE[t]) for t in TEST_CASE ]

In [71]:
[ t for t in TEST_CASE ]

['musthave_dice_model', 'musthave_model', 'musthave_dice_naruki_model']

In [72]:
TEST_CASE_TABLE[0]

,0,1,2,3,4,5,6,7,8,9
0,"(doctoral, 0.924271404743)","(postdoc, 0.921362996101)","(bigdata, 0.916629612446)","(doctor, 0.914591312408)","(hadoop, 0.904930412769)","(dsp, 0.903142571449)","(weka, 0.899703621864)","(data_science, 0.897103011608)","(hdfs, 0.891349434853)","(recommendation, 0.890492320061)"
1,"(css, 0.965053498745)","(css3, 0.957265913486)","(html5, 0.949856162071)","(prototype, 0.949148237705)","(react, 0.94149017334)","(html5/css3/javascript, 0.9389783144)","(html, 0.937718868256)","(user_journey, 0.933208525181)","(cascade_style_sheet, 0.92841565609)","(adobe_aem, 0.928062856197)"
2,"(css, 0.965053498745)","(css3, 0.957265913486)","(html5, 0.949856162071)","(prototype, 0.949148237705)","(react, 0.94149017334)","(html5/css3/javascript, 0.9389783144)","(html, 0.937718868256)","(user_journey, 0.933208525181)","(cascade_style_sheet, 0.92841565609)","(adobe_aem, 0.928062856197)"
3,"(perl, 0.945951700211)","(aws, 0.932041227818)","(tool, 0.93145185709)","(administrator, 0.917977929115)","(stream_audio, 0.910002827644)","(c/c++, 0.907448291779)","(hadoop, 0.905795931816)","(map_reduce, 0.905780076981)","(c, 0.902261018753)","(kinesis, 0.90194529295)"
4,"(css, 0.972095429897)","(ued, 0.966288208961)","(prototype, 0.960716664791)","(xd, 0.960151433945)","(uxpin, 0.958795309067)","(ui_designer, 0.956885993481)","(invision, 0.956589460373)","(template, 0.956371307373)","(user_experience_designer, 0.955363810062)","(framer, 0.954596757889)"
5,"(interaction, 0.972752273083)","(visual, 0.968463301659)","(frontends, 0.968219041824)","(user_experience_designer, 0.967073321342)","(quartz_composer, 0.96668189764)","(invision, 0.965562999249)","(css3, 0.964307188988)","(ui_design, 0.963813662529)","(react, 0.963174343109)","(xd, 0.962453901768)"
6,"(html, 0.972095429897)","(prototype, 0.968522727489)","(javascript, 0.965053498745)","(ued, 0.963989853859)","(invision, 0.961262583733)","(xd, 0.9608720541)","(css3, 0.960782408714)","(html5, 0.960750997066)","(quartz_composer, 0.960707306862)","(ui_designer, 0.959810853004)"
7,"(less, 0.982120513916)","(template, 0.975835263729)","(responsive, 0.970739722252)","(cs, 0.969339609146)","(npm, 0.968224406242)","(frontend, 0.96731364727)","(bower, 0.966849803925)","(dojo, 0.965426206589)","(load, 0.965268015862)","(three, 0.96525144577)"
8,"(npm, 0.986540019512)","(bower, 0.983767449856)","(adobe_aem, 0.983366787434)","(backbone, 0.983056306839)","(prototype, 0.981605648994)","(or_ruby_on_rail, 0.981309950352)","(3d_graphics, 0.981156587601)","(template, 0.98106443882)","(3d_engine, 0.980781197548)","(jsf, 0.980508267879)"


In [73]:
TEST_CASE_TABLE[1]

,0,1,2,3,4,5,6,7,8,9
0,"(data_mine, 0.946587741375)","(log_mine, 0.913221418858)","(parallel, 0.91216814518)","(algorithms, 0.911779403687)","(anomaly_detection, 0.907564878464)","(large_scale, 0.894648551941)","(statistics, 0.89054453373)","(odesign, 0.887861430645)","(mapreduce, 0.87881141901)","(recommendation, 0.870700955391)"
1,"(drupal, 0.966448664665)","(jquery, 0.965409040451)","(less, 0.965108454227)","(behaviour_drive, 0.965004563332)","(bdd, 0.964688897133)","(sass, 0.964326322079)","(bootstrap, 0.963802874088)","(grunt, 0.962328135967)","(jasmine, 0.96202725172)","(mvw, 0.959979176521)"
2,"(drupal, 0.966448664665)","(jquery, 0.965409040451)","(less, 0.965108454227)","(behaviour_drive, 0.965004563332)","(bdd, 0.964688897133)","(sass, 0.964326322079)","(bootstrap, 0.963802874088)","(grunt, 0.962328135967)","(jasmine, 0.96202725172)","(mvw, 0.959979176521)"
3,"(ruby_on_rail, 0.944096446037)","(ror, 0.944012284279)","(engineer_in_test, 0.943315684795)","(blackbox_test, 0.928975045681)","(80211, 0.92416870594)","(ieee_80211, 0.923894822598)","(omnipeek, 0.923828601837)","(https, 0.92302942276)","(single_player, 0.922428548336)","(iperf, 0.92227178812)"
4,"(css, 0.972033441067)","(template, 0.968834638596)","(adobe_experience_management_suite, 0.96435648...","(cq, 0.964027941227)","(css3, 0.963955402374)","(xhtml, 0.962866663933)","(adobe_cq, 0.958325028419)","(aem, 0.956038534641)","(foundation, 0.955864429474)","(css2, 0.955240547657)"
5,"(css3, 0.970298290253)","(aem, 0.953352451324)","(jquery, 0.953019797802)","(less, 0.945244014263)","(css, 0.941391468048)","(adobe_experience_management_suite, 0.94047647...","(template, 0.940237283707)","(angular, 0.939173579216)","(html, 0.937180519104)","(cq, 0.934721469879)"
6,"(html, 0.972033500671)","(jquery, 0.949057996273)","(javascript, 0.94364207983)","(html5, 0.941391468048)","(css3, 0.93936920166)","(template, 0.939226865768)","(adobe_experience_management_suite, 0.9372433424)","(aem, 0.936206579208)","(cq, 0.935415387154)","(sass, 0.935126662254)"
7,"(knockout, 0.993182778358)","(backbone, 0.992327332497)","(ember, 0.989857435226)","(jquerry, 0.988534092903)","(spa, 0.987402558327)","(durandal, 0.986958682537)","(html/css, 0.98613679409)","(three, 0.985328137875)","(foundation, 0.983917832375)","(npm, 0.98370295763)"
8,"(ember, 0.977777481079)","(etl, 0.976830482483)","(webgl, 0.976110219955)","(svg, 0.975852310658)","(responsive/adaptive_design, 0.973373353481)","(spa, 0.972387611866)","(backbone, 0.971573591232)","(npm, 0.971285581589)","(extract, 0.970165967941)","(three, 0.969876766205)"


In [74]:
TEST_CASE_TABLE[2]

,0,1,2,3,4,5,6,7,8,9
0,"(data_scientist, 0.907000541687)","(nlp, 0.893116116524)","(anomaly_detection, 0.871943593025)","(deep_learn, 0.870284318924)","(random_forest, 0.862106204033)","(recommendation, 0.862098574638)","(scala_language, 0.857527852058)","(k__mean, 0.854613780975)","(tensorflow, 0.853815555573)","(time_series_analysis, 0.85380089283)"
1,"(html5, 0.846334934235)","(ajax, 0.837237775326)","(css3, 0.83621609211)","(mvc_architecture, 0.816482722759)","(mvc_frameworks, 0.808582544327)","(web_tool, 0.807993650436)","(oops_concept, 0.806290745735)","(bootstrap, 0.802878439426)","(senior_web_developer, 0.79932641983)","(webdeveloper, 0.797972679138)"
2,"(html5, 0.846334934235)","(ajax, 0.837237775326)","(css3, 0.83621609211)","(mvc_architecture, 0.816482722759)","(mvc_frameworks, 0.808582544327)","(web_tool, 0.807993650436)","(oops_concept, 0.806290745735)","(bootstrap, 0.802878439426)","(senior_web_developer, 0.79932641983)","(webdeveloper, 0.797972679138)"
3,"(ruby, 0.771286308765)","(perl, 0.757723748684)","(relational_databases, 0.739183008671)","(test_frameworks, 0.734647274017)","(map_reduce, 0.731255888939)","(devop, 0.72961807251)","(kinesis, 0.729043483734)","(bigquery, 0.727345228195)","(administrations, 0.72573363781)","(python_programming/scripting_language, 0.7253..."
4,"(xhtml, 0.79220533371)","(html5, 0.790575385094)","(javascript, 0.777606129646)","(web_design, 0.771491289139)","(ajax, 0.769158482552)","(css3, 0.767653107643)","(html_xhtml, 0.756079077721)","(my_sql, 0.754533052444)","(web_tool, 0.753923177719)","(dreamweaver, 0.749688744545)"
5,"(css3, 0.937505722046)","(ui_developer, 0.92447155714)","(bootstrap, 0.923129379749)","(front__end, 0.915638208389)","(css_3, 0.913625717163)","(ui_engineer, 0.906963944435)","(responsive, 0.905551493168)","(html4, 0.904073417187)","(ui_lead, 0.903929889202)","(front_end_engineer, 0.903929233551)"
6,"(html5, 0.856252551079)","(css3, 0.811155676842)","(web_tool, 0.809652686119)","(ui_developer, 0.807193517685)","(bootstrap, 0.805215239525)","(senior_web_developer, 0.802639603615)","(mvc_frameworks, 0.799567103386)","(css_3, 0.795899987221)","(webdeveloper, 0.795441031456)","(ui_developers, 0.792991220951)"
7,"(node, 0.921982288361)","(bootstrap, 0.921577572823)","(backbone, 0.894273757935)","(react, 0.881976783276)","(ui_developer, 0.872991323471)","(html5, 0.869287610054)","(knockout, 0.868739485741)","(javascript_developer, 0.866702198982)","(bower, 0.86581158638)","(full_stack_developer, 0.865062236786)"
8,"(react, 0.938898682594)","(backbone, 0.924705505371)","(mean_stack, 0.924305737019)","(angular, 0.921982288361)","(mean, 0.909098386765)","(bower, 0.905931711197)","(full_stack_developer, 0.902433216572)","(npm, 0.901626110077)","(frontend, 0.901094675064)","(grunt, 0.898154914379)"


In [75]:
musthave_dice_naruki_model.wv.save_word2vec_format("musthave_dice_naruki.model")

2017-05-07 10:36:32,347 : INFO : storing 23483x300 projection weights into musthave_dice_naruki.model


In [76]:
!ls -l

total 146332
-rw-r--r-- 1 root root     1069 13:19 17 Th04 LICENSE
-rw-r--r-- 1 root root 16656175 10:24  7 Th05 musthave_dice_naruki_100d.model
-rw-r--r-- 1 root root 33111241 10:24  7 Th05 musthave_dice_naruki_200d.model
-rw-r--r-- 1 root root 67163623 10:36  7 Th05 musthave_dice_naruki.model
-rw-r--r-- 1 root root  8885546 15:44 16 Th04 mustHaveSkills-2.csv
-rw-r--r-- 1 root root 18240737 11:17 17 Th04 naukri_skill_full
-rw-r--r-- 1 root root  1276232 15:57 16 Th04 prepare_data.ipynb
-rw-r--r-- 1 root root  3972850 15:54 16 Th04 prep_data_tokens_underscore_1
-rw-r--r-- 1 root root       42 13:19 17 Th04 README.md
-rw-r--r-- 1 root root   139130 13:53 17 Th04 skill2vec_ver2.ipynb
-rw-r--r-- 1 root root    32300 10:33  7 Th05 skill2vec_ver3.ipynb
drwxr-xr-x 2 root root       46 13:19 17 Th04 test
-rw-r--r-- 1 root root   246074 08:58 20 Th04 tfidf.ipynb
-rw-r--r-- 1 root root        0 10:04 18 Th04 untitled.txt
drwxr-xr-x 2 root root       63 13:19 17 Th04 word2vec_skills
-rw-r--r-- 1

In [77]:
musthave_dice_naruki_model_100d = word2vec.Word2Vec(musthave_dice_naruki_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
          size=100, 
          min_count=5, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
          iter=4,
          sg =1
    )

musthave_dice_naruki_model_200d = word2vec.Word2Vec(musthave_dice_naruki_data, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
          size=200, 
          min_count=5, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
          iter=4,
          sg =1
    )

2017-05-07 10:36:37,044 : INFO : collecting all words and their counts
2017-05-07 10:36:37,046 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-07 10:36:37,214 : INFO : PROGRESS: at sentence #10000, processed 216479 words, keeping 11609 word types
2017-05-07 10:36:37,266 : INFO : PROGRESS: at sentence #20000, processed 316511 words, keeping 21557 word types
2017-05-07 10:36:37,310 : INFO : PROGRESS: at sentence #30000, processed 403006 words, keeping 30521 word types
2017-05-07 10:36:37,358 : INFO : PROGRESS: at sentence #40000, processed 489603 words, keeping 38336 word types
2017-05-07 10:36:37,407 : INFO : PROGRESS: at sentence #50000, processed 579752 words, keeping 45956 word types
2017-05-07 10:36:37,447 : INFO : PROGRESS: at sentence #60000, processed 666564 words, keeping 52001 word types
2017-05-07 10:36:37,493 : INFO : PROGRESS: at sentence #70000, processed 751300 words, keeping 58857 word types
2017-05-07 10:36:37,533 : INFO : PROGRESS: at 

In [78]:
musthave_dice_naruki_model_100d.wv.save_word2vec_format("musthave_dice_naruki_100d.model")
musthave_dice_naruki_model_200d.wv.save_word2vec_format("musthave_dice_naruki_200d.model")

2017-05-07 10:36:53,535 : INFO : storing 17151x100 projection weights into musthave_dice_naruki_100d.model
2017-05-07 10:36:54,597 : INFO : storing 17151x200 projection weights into musthave_dice_naruki_200d.model


In [79]:
!ls -l
!free -m

total 145924
-rw-r--r-- 1 root root     1069 13:19 17 Th04 LICENSE
-rw-r--r-- 1 root root 16517779 10:36  7 Th05 musthave_dice_naruki_100d.model
-rw-r--r-- 1 root root 32832888 10:36  7 Th05 musthave_dice_naruki_200d.model
-rw-r--r-- 1 root root 67163623 10:36  7 Th05 musthave_dice_naruki.model
-rw-r--r-- 1 root root  8885546 15:44 16 Th04 mustHaveSkills-2.csv
-rw-r--r-- 1 root root 18240737 11:17 17 Th04 naukri_skill_full
-rw-r--r-- 1 root root  1276232 15:57 16 Th04 prepare_data.ipynb
-rw-r--r-- 1 root root  3972850 15:54 16 Th04 prep_data_tokens_underscore_1
-rw-r--r-- 1 root root       42 13:19 17 Th04 README.md
-rw-r--r-- 1 root root   139130 13:53 17 Th04 skill2vec_ver2.ipynb
-rw-r--r-- 1 root root    32300 10:33  7 Th05 skill2vec_ver3.ipynb
drwxr-xr-x 2 root root       46 13:19 17 Th04 test
-rw-r--r-- 1 root root   246074 08:58 20 Th04 tfidf.ipynb
-rw-r--r-- 1 root root        0 10:04 18 Th04 untitled.txt
drwxr-xr-x 2 root root       63 13:19 17 Th04 word2vec_skills
-rw-r--r-- 1

In [98]:
musthave_dice_naruki_model.wv.similar_by_word(skill_transform('javascript'),topn=10)



[(u'html5', 0.8463349342346191),
 (u'ajax', 0.8372377753257751),
 (u'css3', 0.8362160921096802),
 (u'mvc_architecture', 0.8164827227592468),
 (u'mvc_frameworks', 0.8085825443267822),
 (u'web_tool', 0.8079936504364014),
 (u'oops_concept', 0.8062907457351685),
 (u'bootstrap', 0.8028784394264221),
 (u'senior_web_developer', 0.7993264198303223),
 (u'webdeveloper', 0.7979726791381836)]

In [101]:
musthave_dice_naruki_model_200d.wv.similar_by_word(skill_transform('html5'))

[(u'css3', 0.9307963252067566),
 (u'bootstrap', 0.9234713912010193),
 (u'ui_developer', 0.9192823171615601),
 (u'front_end_engineer', 0.9077576398849487),
 (u'front__end', 0.9058327674865723),
 (u'css_3', 0.9039251208305359),
 (u'ui_engineer', 0.9014566540718079),
 (u'adobe_experience_management_suite', 0.9006956815719604),
 (u'html4', 0.9003456234931946),
 (u'user_interface_developer', 0.8981920480728149)]

In [82]:
# [(u'css3', 0.9307963252067566),
#  (u'bootstrap', 0.9234713912010193),
#  (u'ui_developer', 0.9192823171615601),
#  (u'front_end_engineer', 0.9077576398849487),
#  (u'front__end', 0.9058327674865723),
#  (u'css_3', 0.9039251208305359),
#  (u'ui_engineer', 0.9014566540718079),
#  (u'adobe_experience_management_suite', 0.9006956815719604),
#  (u'html4', 0.9003456234931946),
#  (u'user_interface_developer', 0.8981920480728149)]

[(u'visual_design', 0.8798125982284546),
 (u'user_interface_design', 0.8789986371994019),
 (u'ux_design', 0.8746244311332703),
 (u'ui', 0.8447457551956177),
 (u'interaction_designer', 0.8409931063652039),
 (u'interaction_design', 0.8397772908210754),
 (u'user_interaction', 0.8348618745803833),
 (u'wireframe', 0.8340160250663757),
 (u'transform', 0.8149587512016296),
 (u'mobile_design', 0.8148149251937866)]

In [83]:
musthave_dice_naruki_model.similar_by_word(skill_transform('javascript'))

[(u'html5', 0.8463349342346191),
 (u'ajax', 0.8372377753257751),
 (u'css3', 0.8362160921096802),
 (u'mvc_architecture', 0.8164827227592468),
 (u'mvc_frameworks', 0.8085825443267822),
 (u'web_tool', 0.8079936504364014),
 (u'oops_concept', 0.8062907457351685),
 (u'bootstrap', 0.8028784394264221),
 (u'senior_web_developer', 0.7993264198303223),
 (u'webdeveloper', 0.7979726791381836)]

In [84]:
musthave_dice_naruki_model.similar_by_word(skill_transform('node_js'), 300)

[(u'react', 0.9388986825942993),
 (u'backbone', 0.9247055053710938),
 (u'mean_stack', 0.9243057370185852),
 (u'angular', 0.9219822883605957),
 (u'mean', 0.9090983867645264),
 (u'bower', 0.9059317111968994),
 (u'full_stack_developer', 0.9024332165718079),
 (u'npm', 0.9016261100769043),
 (u'frontend', 0.9010946750640869),
 (u'grunt', 0.8981549143791199),
 (u'css/javascript', 0.8971441984176636),
 (u'gulp', 0.8954569697380066),
 (u'bootstrap', 0.8946527242660522),
 (u'mongodb', 0.894255518913269),
 (u'javascript_developer', 0.8926291465759277),
 (u'mean_stack_developer', 0.8926271200180054),
 (u'mongo_db', 0.8904428482055664),
 (u'front_end_web_developer', 0.8894308805465698),
 (u'service_workers', 0.8887876272201538),
 (u'ui_architect', 0.8884862661361694),
 (u'less', 0.888136088848114),
 (u'loopback', 0.8872900009155273),
 (u'firebase', 0.8807092905044556),
 (u'mocha', 0.8802285194396973),
 (u'meanstack', 0.880145788192749),
 (u'front__end', 0.8791353106498718),
 (u'javascript_mvc', 0.8

In [85]:
musthave_dice_naruki_model_100d.similar_by_word(skill_transform('node_js'), 300)

[(u'react', 0.9410043358802795),
 (u'backbone', 0.9235603213310242),
 (u'angular', 0.9139069318771362),
 (u'mean_stack', 0.9107990860939026),
 (u'npm', 0.8981935977935791),
 (u'mean', 0.8961061239242554),
 (u'mongodb', 0.8947668075561523),
 (u'bower', 0.89299476146698),
 (u'gulp', 0.8898729085922241),
 (u'full_stack_developer', 0.8898143768310547),
 (u'less', 0.8879895806312561),
 (u'grunt', 0.8874234557151794),
 (u'bootstrap', 0.8813268542289734),
 (u'javascript_mvc', 0.8796409368515015),
 (u'ui_architect', 0.878745436668396),
 (u'express', 0.8783759474754333),
 (u'javascript_developer', 0.8779905438423157),
 (u'loopback', 0.8772644996643066),
 (u'scss', 0.876909077167511),
 (u'meanstack', 0.8767061233520508),
 (u'fullstack_developer', 0.8745248317718506),
 (u'frontend_developer', 0.8743304014205933),
 (u'oo_javascript', 0.8734292984008789),
 (u'frontend', 0.8726052641868591),
 (u'mean_stack_developer', 0.8722931146621704),
 (u'mocha', 0.8718781471252441),
 (u'ionic', 0.86925601959228

In [86]:
re.sub(r'^angular.*$', 'angular', "angular.js")

'angular'